In [2]:
import threading
import time
import datetime


global last_ts_now
last_ts_now = datetime.datetime.now()
print("last_ts_now", last_ts_now)

global command_buffer
command_buffer = ""

def heartbeat():
    
    import winsound

    def beep():
        frequency = 500  # Set Frequency To 2500 Hertz
        duration = 777  # Set Duration To 1000 ms == 1 second
        winsound.Beep(frequency, duration)

    while True:
        global last_ts_now
        global command_buffer
        
        
        ts_now = datetime.datetime.now()

        time_difference = ts_now - last_ts_now

        # Convert the difference to milliseconds
        seconds = time_difference.total_seconds()

        

        term_key = "<key>Key.alt_l  <key>Key.tab"

        

        if len(command_buffer) > 0 and int(seconds) > 2:
            ts = ts_now.strftime('%Y-%m-%d_%H-%M-%S.%f')
            
            screenshot_file = f"screen_{ts}"
            
            cb = command_buffer.strip()
            if term_key not in cb:

                capture_screenshot(output_dir, screenshot_file)
                logging.info(f"{screenshot_file}\t{str(cb)}")
                print("Snapshot:", last_ts_now, command_buffer, seconds)
            
            beep()

            command_buffer = ""
        time.sleep(0.5)

heartbeat_thread = threading.Thread(target=heartbeat)

heartbeat_thread.start()

# ==========================================================

import pyautogui


def capture_screenshot(out_dir, key):
    # Capture the screenshot
    screenshot = pyautogui.screenshot()

    # Save the screenshot
    screenshot.save(f"{out_dir}/screenshot_{key}.png")


from pynput.keyboard import Key, Listener
import logging



output_dir = f"session_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
print("output_dir", output_dir)


import os

os.makedirs(output_dir)

# Set up logging to log keystrokes to a file
logging.basicConfig(filename=(f"{output_dir}/keylog.txt"), level=logging.DEBUG, format="%(message)s")



def on_press(key):

    s = f"{key}"
    if len(s) > 0 and s[0] == '\'' and s[-1] == '\'':
        s = s[1:-1]

    tag = ""
    if "Key." in s:
        tag = "key"   
        s = f" <{tag}>{s} "         
    
    global command_buffer
    command_buffer += s
    ts_now = datetime.datetime.now()

    global last_ts_now
    last_ts_now = ts_now

# Start listening to keystrokes
with Listener(on_press=on_press) as listener:
    listener.join()

last_ts_now 2024-07-28 15:58:47.013468
output_dir session_2024-07-28_15-58-47
Snapshot: 2024-07-28 15:58:57.776981 git <key>Key.space status <key>Key.enter  3.395388
Snapshot: 2024-07-28 15:59:11.011769 git <key>Key.space branch <key>Key.enter  3.374591
